# 过拟合、欠拟合及其解决方案
## 训练误差&泛化误差
训练误差：模型训练过程中预测结果的误差。
泛化误差：模型在实际运用中预测结果的误差，泛化误差的大小直接关系到模型的好坏，要训练好的模型就要使泛化误差尽可能的小。
训练集：用于训练模型的数据集。
验证集：用于验证模型好坏的数据集。
K折交叉验证：把原始训练数据集划分成K个不重合的子数据集，然后对模型做K次训练和验证，每次用一个子数据集验证模型，其余数据集训练模型，最后对K次的训练误差和泛化误差分别求平均。
# 权重衰减
权重衰减等价于L2范数正则化。正则化是通过给损失函数添加惩罚项来减小模型参数，以应对过拟合。
## L2范数正则化
正则化是给损失函数添加惩罚项，所以L2范数正则化就是给损失函数添加L2范数惩罚项，L2范数惩罚项是各权重参数的平方和与一个正常数的乘积，以线性回归为例，假设线性回归损失函数为：
![Image Name](https://cdn.kesci.com/upload/image/q5qtqkhicj.PNG?imageView2/0/w/960/h/960)
则L2范数正则化后损失函数为：
![Image Name](https://cdn.kesci.com/upload/image/q5qtr1bonl.PNG?imageView2/0/w/960/h/960)
其中λ是大于0的 超参数。可以看出，λ较大时，惩罚项在损失函数中比重较大，会导致权重参数更接近0；λ较小时，惩罚项的作用相对较小。对于绝对值较大的模型参数，权重衰减通过惩罚它们为模型增加了限制，一般对过拟合有效。
## pytorch简洁实现

In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import sys
sys.path.append("/home/kesci/input")
import d2lzh1981 as d2l

print(torch.__version__)

# 初始化模型参数
n_train, n_test, num_inputs = 20, 100, 200
true_w, true_b = torch.ones(num_inputs, 1) * 0.01, 0.05

features = torch.randn((n_train + n_test, num_inputs))
labels = torch.matmul(features, true_w) + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)
train_features, test_features = features[:n_train, :], features[n_train:, :]
train_labels, test_labels = labels[:n_train], labels[n_train:]

# 定义训练和测试
batch_size, num_epochs, lr = 1, 100, 0.003
net, loss = d2l.linreg, d2l.squared_loss

dataset = torch.utils.data.TensorDataset(train_features, train_labels)
train_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)

1.3.0


In [2]:
def fit_and_plot_pytorch(wd):
    # 对权重参数衰减。权重名称一般是以weight结尾
    net = nn.Linear(num_inputs, 1)
    nn.init.normal_(net.weight, mean=0, std=1)
    nn.init.normal_(net.bias, mean=0, std=1)
    optimizer_w = torch.optim.SGD(params=[net.weight], lr=lr, weight_decay=wd) # 对权重参数衰减
    optimizer_b = torch.optim.SGD(params=[net.bias], lr=lr)  # 不对偏差参数衰减
    
    train_ls, test_ls = [], []
    for _ in range(num_epochs):
        for X, y in train_iter:
            l = loss(net(X), y).mean()
            optimizer_w.zero_grad()
            optimizer_b.zero_grad()
            
            l.backward()
            
            # 对两个optimizer实例分别调用step函数，从而分别更新权重和偏差
            optimizer_w.step()
            optimizer_b.step()
        train_ls.append(loss(net(train_features), train_labels).mean().item())
        test_ls.append(loss(net(test_features), test_labels).mean().item())
    d2l.semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'loss',
                 range(1, num_epochs + 1), test_ls, ['train', 'test'])
    print('L2 norm of w:', net.weight.data.norm().item())

In [3]:
fit_and_plot_pytorch(0)

L2 norm of w: 12.73517894744873


<Figure size 252x180 with 1 Axes>

In [4]:
fit_and_plot_pytorch(3)

L2 norm of w: 0.038453906774520874


<Figure size 252x180 with 1 Axes>

# 倒置丢弃法
丢弃法是深度学习模型常用的应对过拟合的方法。这里笔记倒置丢弃法。
先看一个多层感知机：
![Image Name](https://cdn.kesci.com/upload/image/q5qyh2yq89.png?imageView2/0/w/960/h/960)
这个感知机有一个隐藏层，对这个隐藏层使用丢弃法就是随机将该层中几个神经元丢弃。使用了丢弃法后此感知机可能变成这个样子：
![Image Name](https://cdn.kesci.com/upload/image/q5qyhksdfc.png?imageView2/0/w/960/h/960)
假设丢弃概率为p，那么hi被丢弃的概率就是p，而hi剩下1-p的概率会除以1-p进行拉伸。假设随机变量ri为0或1，为0的概率为p，为1的概率为1-p，则用丢弃法计算新的隐藏单元hi'为：
![Image Name](https://cdn.kesci.com/upload/image/q5qyi9io8q.PNG?imageView2/0/w/960/h/960)
这里除以1-p是为了使新隐藏单元的期望不变：
![Image Name](https://cdn.kesci.com/upload/image/q5qyihisnq.PNG?imageView2/0/w/960/h/960)
## pytorch简洁实现

In [5]:
# 准备工作
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import sys
sys.path.append("/home/kesci/input")
import d2lzh1981 as d2l

print(torch.__version__)

# 参数的初始化
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256

W1 = torch.tensor(np.random.normal(0, 0.01, size=(num_inputs, num_hiddens1)), dtype=torch.float, requires_grad=True)
b1 = torch.zeros(num_hiddens1, requires_grad=True)
W2 = torch.tensor(np.random.normal(0, 0.01, size=(num_hiddens1, num_hiddens2)), dtype=torch.float, requires_grad=True)
b2 = torch.zeros(num_hiddens2, requires_grad=True)
W3 = torch.tensor(np.random.normal(0, 0.01, size=(num_hiddens2, num_outputs)), dtype=torch.float, requires_grad=True)
b3 = torch.zeros(num_outputs, requires_grad=True)

params = [W1, b1, W2, b2, W3, b3]

drop_prob1, drop_prob2 = 0.2, 0.5

num_epochs, lr, batch_size = 5, 100.0, 256  # 这里的学习率设置的很大，原因与之前相同。
loss = torch.nn.CrossEntropyLoss()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, root='/home/kesci/input/FashionMNIST2065')

1.3.0


In [6]:
# 网络搭建
net = nn.Sequential(
        d2l.FlattenLayer(),
        nn.Linear(num_inputs, num_hiddens1),
        nn.ReLU(),
        nn.Dropout(drop_prob1),
        nn.Linear(num_hiddens1, num_hiddens2), 
        nn.ReLU(),
        nn.Dropout(drop_prob2),
        nn.Linear(num_hiddens2, 10)
        )

for param in net.parameters():
    nn.init.normal_(param, mean=0, std=0.01)

In [7]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)

epoch 1, loss 0.0044, train acc 0.559, test acc 0.703
epoch 2, loss 0.0023, train acc 0.781, test acc 0.743
epoch 3, loss 0.0019, train acc 0.820, test acc 0.805
epoch 4, loss 0.0018, train acc 0.836, test acc 0.819
epoch 5, loss 0.0016, train acc 0.848, test acc 0.774
